In [ ]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import csv

# 주의!! inception 폴더가 현재 디렉토리에 있어야 정상 작동한다. 
from inception.data import build_image_data
from inception import image_processing
from inception import inception_model as inception
from os import listdir
from os.path import isfile, join

# weight파일의 경로
checkpoint_dir = "D:\Haeri_Data\Programming\JupyterNotebook\ImageSearch\inception-v3"

# 한번에 몇개의 이미지를 프로세싱할지
batch_size = 100

# 이미지의 경로
my_image_path = "D:\Haeri_Data\Programming\JupyterNotebook\ImageSearch\images/"

# 이미지 폴더를 스캔해서 리스트로 만든다. 
img_file_list = [f for f in listdir(my_image_path)  if (f.rfind('jpg') > -1)]
file_size = len(img_file_list)

# 생성한 이미지 리스트를 csv파일로 저장한다. 
pd.DataFrame(img_file_list).to_csv("img_file_list.csv", header=False, index=False)

# 이미지를 분석해 데이터를 만드는 함수를 정의한다. 
# 이미지가 여러개일 경우
def inference_on_multi_image():
  print("total image size {} ".format(file_size) )
  
  total_batch_size = file_size // batch_size + 1
  infdata_list = []

  for n in range(total_batch_size):
      print("step :{} / {}".format(n + 1, total_batch_size))
      mini_batch = img_file_list[n * batch_size: (n + 1) * batch_size]
      mini_adarr = np.ndarray(shape=(0, 299,299,3))
        
      with tf.Graph().as_default():
        num_classes = 1001 # (logit size)

        coder = build_image_data.ImageCoder()
        for i, image in enumerate(mini_batch):
          image_buffer, _, _ =  build_image_data._process_image(my_image_path + image, coder)
          image = image_processing.image_preprocessing(image_buffer, 0, False) # image -> (299, 299, 3)
          image = tf.expand_dims(image, 0) # (299, 299,3) -> (1, 299, 299, 3)
          mini_adarr = tf.concat([mini_adarr, image], 0) 

        logits, _ = inception.inference(mini_adarr, num_classes, for_training=False, restore_logits=True)

        saver = tf.train.Saver()
        with tf.Session() as tf_session:
          ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
          if ckpt and ckpt.model_checkpoint_path:
            if os.path.isabs(ckpt.model_checkpoint_path):
              # Restores from checkpoint with absolute path.
              saver.restore(tf_session, ckpt.model_checkpoint_path)
            else:
              # Restores from checkpoint with relative path.
              saver.restore(tf_session, os.path.join(checkpoint_dir,
                                               ckpt.model_checkpoint_path))

          l = tf_session.run([logits])
          for row in l[0]:
            infdata_list.append(row)
                
  return infdata_list

# 확보한 이미지의 분석 데이터를 만든다. 
# 굉장히 오래 걸린다. 
infdata_list = inference_on_multi_image()

# list를 csv파일로 저장한다. 
csvfile = open("infdata_list.csv", "w", newline="")
csvwriter = csv.writer(csvfile)
for row in logit_list:
    csvwriter.writerow(row)
    
csvfile.close()